## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## Compute the camera calibration using chessboard images

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt

import ipywidgets as wg
from IPython.display import display

from ipywidgets import interact, interactive, fixed

%matplotlib qt

In [ ]:


fig, axs = plt.subplots(4,5, figsize=(16, 11))
axs=axs.ravel()
 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
cnt=0
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)
        #img = cv2.cvtColor(img, cv2.cv.CV_BGR2RGB)
        #plt.imshow(img)
        #plt.show()
        axs[cnt-1].imshow(img)
    cnt+=1


#cv2.destroyAllWindows()

In [ ]:
# performs the camera calibration, image distortion correction and 
# returns the undistorted image
def cal_undistort(img, objpoints, imgpoints):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
    undist = cv2.undistort(img, mtx, dist, None, mtx) 
    return undist




In [ ]:
# Read in an image
img = cv2.imread('./camera_cal/calibration1.jpg')



undistorted = cal_undistort(img, objpoints, imgpoints)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(cv2.cvtColor(undistorted, cv2.COLOR_BGR2RGB))
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


#TODO Save camera callibration parameters to pickle file

### Example of a distortion-corrected image.

In [ ]:
# Read in an image
img = cv2.imread('./test_images/test2.jpg')



undistorted = cal_undistort(img, objpoints, imgpoints)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(cv2.cvtColor(undistorted, cv2.COLOR_BGR2RGB))
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


#TODO Save camera callibration parameters to pickle file

## Perspective Transform

In [ ]:
def image_unwarp(img, src, dst):
    
    undist = cal_undistort(img, objpoints, imgpoints)
        
    # Convert undistorted image to grayscale
    gray = cv2.cvtColor(undist, cv2.COLOR_BGR2GRAY)
        
    img_size = (gray.shape[1], gray.shape[0]) 
        
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    
    
    #Inverse Perspective Transform 
    Minv = cv2.getPerspectiveTransform(dst, src)
    
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(undist, M, img_size)
    
    return warped, M, Minv


def visualize_pts(img, pts):
    for pt in pts:
        print(tuple(pt))
        img = cv2.circle(img,tuple(pt),10,(255,0,0),-1)
        
    return img

# [ref: http://bistr-o-mathik.org/2012/06/13/simple-transparency-in-opencv/]
def fill_lane(img, pts):
    overlay = img.copy()
    cv2.fillPoly(overlay,np.int_([pts]),(0,255,0))
    cv2.addWeighted(overlay,0.4,img,1-0.4,0,img)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    



In [ ]:

img = cv2.imread('./test_images/test2.jpg')
#cv2.imshow('img',img)

#%matplotlib notebook

#cv2.imshow('img:',img)
#plt.imshow(img)

In [ ]:
h,w=img.shape[:2]
# define source and destination points for transform
src = np.float32([(595,464),
                  (727,464), 
                  (278,682), 
                  (1069,682)])
dst = np.float32([(450,0),
                  (w-450,0),
                  (450,h),
                  (w-450,h)])


unwarped_image, M, Minv = image_unwarp(img,src,dst)


img = visualize_pts(img,src)


pts=np.array([[595,464],[727,464],[1069,682],[278,682]], np.int32)
pts = pts.reshape((-1,1,2))
#plt.imshow(cv2.fillPoly(img,np.int_([pts]),(0,255,0)))
plt.imshow(fill_lane(img,pts))
plt.show()

In [ ]:
plt.imshow(cv2.cvtColor(unwarped_image, cv2.COLOR_BGR2RGB))
plt.show()

## Color Spaces and Gradient Thresholds

In [ ]:


#Region of Interest Mask from Project 1
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image




#Helper function to apply different Image thresholding methods on test images

def apply_method(thresh_method,img_list=None, p_transform=False, call_roi=False,lane_detect_sws=False, skip_sws=True, highlight_lane=False):
    
    images_path=[]
    images=[]
    original=[]
    processed=[]
    if(img_list == None):
        # Make a list of calibration images
        images_path = glob.glob('./test_images/*.jpg')
        for fname in images_path:
            img = cv2.imread(fname)
            image = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            images.append(image)
    else:
        
        images = img_list
   
    
    fig, axs = plt.subplots(len(images),2, figsize=(80, 142))
    axs=axs.ravel()

    #fig.suptitle(str(thresh_method), fontsize=14, fontweight='bold')
    # Step through the list and search for chessboard corners
    cnt=0
    
    for image in images:
       
        undistorted = cal_undistort(image, objpoints, imgpoints)
        
        original_image = undistorted
        if p_transform == True:
            original_image, M, Minv = image_unwarp(original_image,src,dst)
       
                        

        if highlight_lane == True:
            axs[cnt].imshow(undistorted)
            original.append(undistorted)
        else: 
            axs[cnt].imshow(original_image)
            original.append(original_image)
        cnt+=1
        
        processed_image=thresh_method(original_image)
        if lane_detect_sws == True:
            processed_image_fitted, left_fit, right_fit, left_fitx, right_fitx, leftx, lefty, rightx, righty, ploty = perform_lane_sws(processed_image)
            #print("***** len lefx={} len rightx={}".format(len(leftx),len(rightx)))
            #r1,r2 = measure_curvature3(ploty, left_fit, right_fit, leftx, rightx)
            #print("***** r1={} r2={}".format(r1,r2))
            if skip_sws ==  True:
                processed_image_fitted, left_fit, right_fit, left_fitx, right_fitx, leftx, lefty, rightx, righty, ploty = lane_detect_no_sws(processed_image,left_fit, right_fit)
            
            if highlight_lane == True:
                result=highlight_detected_lane(undistorted,original_image,Minv,ploty, left_fitx,right_fitx,undistorted)
                processed_image_fitted = result
            
            processed_image = processed_image_fitted
            axs[cnt].imshow(processed_image_fitted)
        else:
            if call_roi == True:
                imshape = processed_image.shape
                vertices = np.array([[(0,imshape[0]),(600, 440), (720, 440), (imshape[1],imshape[0])]], dtype=np.int32)
                processed_image = region_of_interest(processed_image,vertices)
            axs[cnt].imshow(processed_image, cmap='gray')
        processed.append(processed_image)
        cnt+=1
    return original, processed

#apply_and_show()

### Sobel

In [ ]:
# Define a function that applies Sobel x or y, 
# then takes an absolute value and applies a threshold.

def abs_sobel_thresh(img, orient='x', thresh_min=15, thresh_max=255):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Return the result
    return binary_output

In [ ]:


original_list, processed_list = apply_method(abs_sobel_thresh)



### Sobel Magnitude Threshold

In [ ]:
# Define a function that applies Sobel x and y, 
# then computes the magnitude of the gradient
# and applies a threshold
def magnitude_thresh(img, sobel_kernel=9, mag_thresh=(60, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output

In [ ]:
original_list, processed_list =apply_method(magnitude_thresh)


### Direction of the Gradient

In [ ]:
# Define a function that applies Sobel x and y, 
# then computes the direction of the gradient
# and applies a threshold.
# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=9, thresh=(0.7, 1.1)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

In [ ]:
original_list, processed_list = apply_method(dir_threshold)

### Combined Magnitude and Direciton Threshold

###  S Channel Threshold (HLS)

In [ ]:

def hls_select(img, thresh=(60, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    return binary_output


def color_thresholds(img, HLS_s_threshold=(60, 255), HSV_v_threshold=(120,255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel > HLS_s_threshold[0]) & (s_channel <= HLS_s_threshold[1])] = 1
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    v_channel = hsv[:,:,2]
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel > HSV_v_threshold[0]) & (v_channel <= HSV_v_threshold[1])] = 1  
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_binary == 1) & (v_binary == 1)] = 1
    return binary_output

In [ ]:
original_list, processed_list = apply_method(color_thresholds)

### Combined S Channel and Gradient Threhold 

In [ ]:
def combine_thresholds(img):
    sobel_x = abs_sobel_thresh(img, orient='x')
    sobel_y = abs_sobel_thresh(img, orient='y')
    mag_binary = magnitude_thresh(img)
    dir_binary = dir_threshold(img)
    s_color_img = color_thresholds(img)
    
    combined = np.zeros_like(dir_binary)
    combined[(sobel_x == 1) & (sobel_y == 1) & (mag_binary == 1) | (s_color_img == 1) | (mag_binary == 1) & (dir_binary == 1)] = 1
    
    return combined


    

In [ ]:
original_list, processed_list = apply_method(combine_thresholds)

### Combine Region of Intrest

In [ ]:

original_list, processed_list = apply_method(combine_thresholds,call_roi=True)
masked_list = original_list

## Combine Perspective Transform with Image thresholding

In [ ]:
original_list,processed_list=apply_method(combine_thresholds,img_list=masked_list,p_transform=True)
masked_perspective_list = processed_list

### Finding the Lane (Sliding Window Search)

In [ ]:
binary_warped=processed_list[0]
plt.imshow(binary_warped, cmap='gray')

In [ ]:
# Assuming you have created a warped binary image called "binary_warped"
# Take a histogram of the bottom half of the image
histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)
plt.plot(histogram)

In [ ]:
# Lane Sliding Window Search (SWS)
def perform_lane_sws(binary_warped):
    histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)
    #plt.plot(histogram)
    # Create an output image to draw on and  visualize the result
    #out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    out_img = np.array(cv2.merge((binary_warped,binary_warped,binary_warped)),np.uint8)
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 10
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 50
    # Set minimum number of pixels found to recenter window
    minpix = 1
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    #plt.imshow(out_img)
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    #plt.xlim(0, 1280)
    #plt.ylim(720, 0)
    
    return out_img, left_fit, right_fit, left_fitx, right_fitx, leftx, lefty, rightx, righty, ploty
         



### Skip the sliding windows  

In [ ]:

def lane_detect_no_sws(binary_warped, left_fit, right_fit):
    # Assume you now have a new warped binary image 
    # from the next frame of video (also called "binary_warped")
    # It's now much easier to find line pixels!
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
    left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
    right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Create an image to draw on and an image to show the selection window
    out_img = np.uint8(np.dstack((binary_warped, binary_warped, binary_warped))*255)
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                                  ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                                  ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    plt.imshow(result)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    
    return result, left_fit, right_fit, left_fitx, right_fitx, leftx, lefty, rightx, righty, ploty

In [ ]:
def measure_curvature(ploty, leftx, rightx):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    y_eval = np.max(ploty)
    
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(leftx*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(rightx*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    #print(left_curverad, 'm', right_curverad, 'm')
    return left_curverad,right_curverad



def measure_curvature2(ploty, left_fit, right_fit, leftx, rightx):
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    y_eval = np.max(ploty)
    #leftx = np.array(leftx, dtype=np.float32)
    #rightx = np.array(rightx, dtype=np.float32)
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    left_curverad = ((1 + (2*left_fit[0]*y_eval*ym_per_pix + left_fit[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval*ym_per_pix + right_fit[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    

    
    return left_curverad, right_curverad


def measure_curvature3(ploty, left_fit, right_fit, leftx, rightx):

    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    return left_curverad, right_curverad

In [ ]:
original_list,processed_list=apply_method(combine_thresholds,p_transform=True,lane_detect_sws=True,skip_sws=False)

In [ ]:
original_list,processed_list=apply_method(combine_thresholds,p_transform=True,lane_detect_sws=True,skip_sws=True)

In [ ]:
def highlight_detected_lane(original, warped, Minv, ploty, left_fitx, right_fitx, undist):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    #color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    color_warp = original.copy()
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    #print([pts])
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (original.shape[1], original.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    return result

def lane_mapping(img_shape, warped, Minv, ploty, left_fitx, right_fitx, undistorted):
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    print(len(left_fitx))
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255,0)) 
    newwarp = cv2.warpPerspective(color_warp, Minv, (img_shape[1], img_shape[0])) 
    out_img = cv2.addWeighted(undistorted, 1, newwarp, 0.3, 0)
    return out_img

In [ ]:



img = cv2.imread('./test_images/test2.jpg')
#cv2.imshow('img',img)

#%matplotlib notebook

plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

undistorted_img= cal_undistort(img, objpoints, imgpoints)


warped_image, M, Minv = image_unwarp(img,src,dst)

binary_warped=combine_thresholds(warped_image)

plt.imshow(binary_warped,cmap='gray')


out_img, left_fit, right_fit, left_fitx, right_fitx, leftx, lefty, rightx, righty, ploty = perform_lane_sws(binary_warped)
         
highlight_detected_lane(img, warped_image, Minv, ploty, left_fitx, right_fitx, undistorted_img)   

#new_img, ploty, left_fitx, right_fitx, left_fit, right_fit, leftx, rightx, leftx_base, rightx_base = sliding_windows(warped_image)
#out = lane_mapping(img.shape[:2],warped_image,Minv,ploty, left_fitx,right_fitx, cv2.cvtColor(undistorted_img, cv2.COLOR_BGR2RGB))
print(len(left_fitx))

plt.imshow(out)



In [ ]:
original_list,processed_list=apply_method(combine_thresholds,p_transform=True,lane_detect_sws=True,skip_sws=False,highlight_lane=True)

In [ ]:

def pipeline(img):
    
    # Undistort the image
    undist = cal_undistort(img, objpoints, imgpoints)
    
    # perform perspective transform
    unwarp = image_unwarp(undist, src, dst)
    
    piped_image = unwarp
    return piped_image